In [ ]:
import json
import re
import os
import pandas as pd

from langchain_ollama import ChatOllama

from langchain.messages import AIMessage

import yfinance as yf

import OnDeviceAgent.ChatApi.finance_tools as ft

In [ ]:
ticker = yf.Ticker("MSFT")
ticker.get_cashflow()

,2025-06-30,2024-06-30,2023-06-30,2022-06-30,2021-06-30
FreeCashFlow,7.161100e+10,7.407100e+10,5.947500e+10,6.514900e+10,NaN
RepurchaseOfCapitalStock,-1.842000e+10,-1.725400e+10,-2.224500e+10,-3.269600e+10,NaN
RepaymentOfDebt,-3.216000e+09,-2.907000e+10,-2.750000e+09,-9.023000e+09,NaN
IssuanceOfDebt,0.000000e+00,2.439500e+10,0.000000e+00,0.000000e+00,NaN
IssuanceOfCapitalStock,2.056000e+09,2.002000e+09,1.866000e+09,1.841000e+09,NaN
CapitalExpenditure,-6.455100e+10,-4.447700e+10,-2.810700e+10,-2.388600e+10,NaN
EndCashPosition,3.024200e+10,1.831500e+10,3.470400e+10,1.393100e+10,NaN
BeginningCashPosition,1.831500e+10,3.470400e+10,1.393100e+10,1.422400e+10,NaN
EffectOfExchangeRateChanges,6.300000e+07,-2.100000e+08,-1.940000e+08,-1.410000e+08,NaN
ChangesInCash,1.186400e+10,-1.617900e+10,2.096700e+10,-1.520000e+08,NaN


In [10]:
model_350m = "granite4:350m"
model_1b = "granite4:1b"
model_3b = "granite4:3b"

In [11]:
n_cores = os.cpu_count()
n_cores

8

In [12]:
llm_350m = ChatOllama(
    model=model_350m,
    num_thread=n_cores,  # CPU threads
    temperature=0.0
).bind_tools([ft.get_key_financial_metrics, ft.get_balance_sheet, ft.get_dividends, ft.get_latest_news])

llm_1b = ChatOllama(
    model=model_1b,
    num_thread=n_cores,  # CPU threads
    temperature=0.0
)#.bind_tools([ft.get_balance_sheet, ft.get_dividends])

llm_3b =  ChatOllama(
    model=model_3b,
    num_thread=n_cores,  # CPU threads
    temperature=0.0
)#.bind_tools([ft.get_balance_sheet, ft.get_dividends])

In [ ]:
# change input text as desired
chat = [
    {
        "role": "system",
        "content": """You are a financial analysis assistant who has access to various tools for retrieving stock market and financial data about specific stocks.

            Use these tools to assist with user queries about stock performance, historical data and financial data.
        """
    },
    {
        "role": "user", 
        "content": "Can you summarise the lastest news for Nvidia?" 
    }
]
result = llm_350m.invoke(chat)
print(result.content)

In [7]:
if isinstance(result, AIMessage) and result.tool_calls:
    print(result.tool_calls)


[{'name': 'get_latest_news', 'args': {'ticker': 'NVDA'}, 'id': 'c7ef911c-6e9c-4473-9b63-ab1fde5e3b15', 'type': 'tool_call'}]


In [9]:
# Tool mapping
tool_mapping = {
    "get_balance_sheet": ft.get_balance_sheet,
    "get_dividends": ft.get_dividends,
    "get_key_financial_metrics": ft.get_key_financial_metrics,
    "get_latest_news": ft.get_latest_news
}

In [10]:
result.tool_calls[0]["name"]

'get_latest_news'

In [11]:
try:
    tool_response = tool_mapping[result.tool_calls[0]["name"]].invoke(result.tool_calls[0]["args"])
    # tool_func = tool_mapping[result.tool_calls[0]["name"]]
    # result = tool_func(**result.tool_calls[0]["args"])
except Exception as e:
    result = {"error": str(e)}
tool_response

'[\n  {\n    "id": "850b63c2-d571-49b7-a7bb-7296fe16fe9c",\n    "title": "Economic data returns, retail earnings feature in holiday-shortened week: What to watch this week",\n    "summary": "Investors will be tracking retail earnings and economic data as the government works through a backlog of reports delayed by the shutdown.",\n    "pubDate": "2025-11-23T12:37:46Z",\n    "provider": "Yahoo Finance",\n    "contentType": "STORY"\n  },\n  {\n    "id": "8d419152-1254-303b-b245-e33cbbfa30b6",\n    "title": "Dow Jones Futures Due, Bitcoin Bounces; Nvidia, Apple, Eli Lilly In Focus",\n    "summary": "Futures loom as bitcoin, a possible catalyst for stock market woes, rebounded over the weekend. Watch Nvidia, Apple and Eli Lilly.",\n    "pubDate": "2025-11-23T19:02:37Z",\n    "provider": "Investor\'s Business Daily",\n    "contentType": "STORY"\n  },\n  {\n    "id": "6aed1df1-59a2-3bbe-9f21-7d7265d7836b",\n    "title": "Trump Administration Ponders Allowing Nvidia To Trade H200 AI Chips Wit

In [12]:
chat.append({
    "role": "tool",
    "tool_call_id": result.tool_calls[0]["id"],
    "name": result.tool_calls[0]["name"],
    "content": tool_response
})
chat

[{'role': 'system',
  'content': '\n            You are a financial analysis assistant who has access to various tools for retrieving stock market and financial data about specific stocks.\n\n            Use these tools to assist with user queries about stock performance, historical data and financial data.\n        '},
 {'role': 'user', 'content': 'Can you summarise the lastest news for Nvidia?'},
 {'role': 'tool',
  'tool_call_id': 'c7ef911c-6e9c-4473-9b63-ab1fde5e3b15',
  'name': 'get_latest_news',
  'content': '[\n  {\n    "id": "850b63c2-d571-49b7-a7bb-7296fe16fe9c",\n    "title": "Economic data returns, retail earnings feature in holiday-shortened week: What to watch this week",\n    "summary": "Investors will be tracking retail earnings and economic data as the government works through a backlog of reports delayed by the shutdown.",\n    "pubDate": "2025-11-23T12:37:46Z",\n    "provider": "Yahoo Finance",\n    "contentType": "STORY"\n  },\n  {\n    "id": "8d419152-1254-303b-b245

In [13]:
result = llm_1b.invoke(chat)
print(result.content)

Here is a summary of the latest news for Nvidia:

1. **Economic Data and Retail Earnings**: Investors will be closely monitoring retail earnings and economic data as the government processes reports delayed by the shutdown.

2. **Bitcoin's Rebound**: Bitcoin rebounded over the weekend, potentially impacting stock market performance. Key stocks to watch include Nvidia, Apple, and Eli Lilly.

3. **Trump Administration's Potential Policy Change**: The Trump administration is reportedly considering allowing Nvidia to trade its H200 AI chips with China, marking a significant shift from previous export control policies.
